In [ ]:
#open the dataset

import pandas as pd
df = pd.read_json(r"//content//Arts_Crafts_and_Sewing_5.json",lines=True)
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"03 29, 2016",AIE8N9U317ZBM,0449819906,{'Format:': ' Kindle Edition'},"Zelmira, Ph.D.",Contains some interesting stitches.,Four Stars,1459209600,NaN,NaN
1,5,True,"08 12, 2015",A3ECOW0TWLH9V6,0449819906,{'Format:': ' Paperback'},Dangerous when Cooking,I'm a fairly experienced knitter of the one-co...,My current favorite go-to guide for inspiration,1439337600,18,NaN
2,4,True,"04 5, 2015",A278N8QX9TY2OS,0449819906,{'Format:': ' Paperback'},Just us,Great book but the index is terrible. Had to w...,"lots of great examples, good instructions, col...",1428192000,3,NaN
3,5,True,"10 11, 2014",A123W8HIK76XCN,0449819906,{'Format:': ' Kindle Edition'},Amazon Customer,I purchased the Kindle edition which is incred...,Another little gem by Melissa Leapman,1412985600,NaN,NaN
4,5,True,"05 8, 2014",A2A6MZ2QB4AE0L,0449819906,{'Format:': ' Paperback'},Sustainability,Very well laid out and very easy to read.\n\nT...,Very comprehensive,1399507200,NaN,NaN


**Getting he Bing Liu lexicon**

In [ ]:
#Get the Bing Liu lexicon

from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))

print('Examples of positive words in opinion lexicon',opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])

print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.


**Creating the Dictionary**

In [ ]:
#Create the “Dictionary”
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
  word_dict[word] = pos_score

# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
  word_dict[word] = neg_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Function to do math**

In [ ]:
#Function to do the math
def bing_liu_score(text):
  sentiment_score = 0
  bag_of_words = word_tokenize(text.lower())
  for word in bag_of_words:
    if word in word_dict:
      sentiment_score += word_dict[word]
      return sentiment_score

In [ ]:
#Make sure all texts are non-empty and aretype text then call the function

df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

**Output of the code**

In [ ]:
#Let’s see the output
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1.0
1,5,I'm a fairly experienced knitter of the one-co...,1.0
2,4,Great book but the index is terrible. Had to w...,1.0
3,5,I purchased the Kindle edition which is incred...,1.0
4,5,Very well laid out and very easy to read.\n\nT...,1.0
5,5,"Beginning her career as a freelance knitter, M...",1.0
6,5,This is a terrific stitch handbook (and I have...,1.0
7,4,The book needs to be coil bound. The content i...,1.0
8,5,I really am enjoying this book! I like the siz...,1.0
9,5,Just received this book and looked over it cov...,1.0


**Checking the Sentiment**

In [ ]:
#One more thing:
#• Does the sentiment count match the “rating” given by thecustomers?

df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.139887
2,0.174485
3,0.458573
4,0.770115
5,0.888414
